# config

> Configuring nbdev and bootstrapping notebook export

- order: 1

In [ ]:
#| default_exp config

In [ ]:
#| export
from datetime import datetime
from fastcore.docments import *
from fastcore.utils import *
from fastcore.meta import *
from fastcore.script import *
from fastcore.style import *
from fastcore.xdg import *

import ast,warnings,tomli
from IPython.display import Markdown
from execnb.nbio import read_nb,NbCell
from urllib.error import HTTPError

In [ ]:
#| hide
from fastcore.test import *
import tempfile

## Configuring nbdev

nbdev is heavily customizeable, thanks to the configuration system defined in this module. There are 2 ways to interact with nbdev's config:

- **In the terminal:** `nbdev_create_config` creates a `pyproject.toml` (if you're starting a new project use `nbdev_new` instead)
- **In your library:** `get_config` returns a `ConfigToml` object.

Read on for more about how these work.

### Creating a config file -

In [ ]:
#| export
pyproject_nm = 'pyproject.toml'
_nbdev_home_dir = 'nbdev'
_user_cfg_name = 'config.toml'

In [ ]:
#| export
def _git_repo():
    try: return repo_details(run('git config --get remote.origin.url'))[1]
    except OSError: return

In [ ]:
#| hide
test_eq(_git_repo(), 'nbdev')
with tempfile.TemporaryDirectory() as d, working_directory(d): test_is(_git_repo(), None)

In [ ]:
#| export
def _get_info(owner, repo, default_branch='main', default_kw='nbdev'):
    from ghapi.all import GhApi
    api = GhApi(owner=owner, repo=repo, token=os.getenv('GITHUB_TOKEN'))
    
    try: r = api.repos.get()
    except HTTPError:
        msg= [f"""Could not access repo: {owner}/{repo} to find your default branch - `{default_branch}` assumed.
Edit `pyproject.toml` if this is incorrect.
In the future, you can allow nbdev to see private repos by setting the environment variable GITHUB_TOKEN as described here:
https://nbdev.fast.ai/api/release.html#setup"""]
        print(''.join(msg))
        return default_branch,default_kw,''
    
    return r.default_branch, default_kw if not getattr(r, 'topics', []) else ' '.join(r.topics), r.description

In [ ]:
#| hide
if os.getenv('GITHUB_ACTIONS') != 'true': # GITHUB_TOKEN in actions has limited scope.
    _branch, _tags, _descrip = _get_info('fastai', 'fastai')
    test_eq(_tags, 'colab deep-learning fastai gpu machine-learning notebooks python pytorch')
    test_eq(_branch, 'main')
    test_eq(_descrip, 'The fastai deep learning library')

In [ ]:
#| export
def _fetch_from_git(raise_err=False):
    "Get information for pyproject.toml from git."
    res={}
    try:
        url = run('git config --get remote.origin.url')
        res['user'],res['repo'] = repo_details(url)
        res['branch'],res['keywords'],desc = _get_info(owner=res['user'], repo=res['repo'])
        if desc: res['description'] = desc
        res['author'] = run('git config --get user.name').strip()
        res['author_email'] = run('git config --get user.email').strip()
    except OSError as e:
        if raise_err: raise(e)
    else: res['lib_name'] = res['repo'].replace('-','_')
    return res

In [ ]:
#| hide
#test_eq(_fetch_from_git(raise_err=True)['lib_name'], 'nbdev')

In [ ]:
#| export
pyproject_tmpl = '''[build-system]
requires = ["setuptools>=64"]
build-backend = "setuptools.build_meta"

[project]
name = "{name}"
dynamic = ["version"]
description = "{description}"
readme = "README.md"
requires-python = ">={min_python}"
license = {{text = "{license}"}}
authors = [{{name = "{author}", email = "{author_email}"}}]
keywords = {keywords}
classifiers = [
    "Programming Language :: Python :: 3",
    "Programming Language :: Python :: 3 :: Only",
]
dependencies = []

[project.urls]
Repository = "{git_url}"
Documentation = "{doc_url}"

[project.entry-points.nbdev]
{lib_path} = "{lib_path}._modidx:d"

[tool.setuptools.dynamic]
version = {{attr = "{lib_path}.__version__"}}

[tool.setuptools.packages.find]
include = ["{lib_path}"]

[tool.nbdev]
'''

In [ ]:
#| export
@call_parse
def nbdev_create_config(
    repo:str=None, # Repo name
    branch:str='main', # Repo default branch
    user:str=None, # Repo username
    author:str=None, # Package author's name
    author_email:str=None, # Package author's email address
    description:str='', # Short summary of the package
    path:str='.', # Path to create config file
    min_python:str='3.10', # Minimum Python version
    license:str='Apache-2.0', # License (SPDX identifier)
):
    "Create a pyproject.toml config file."
    path = Path(path)
    path.mkdir(exist_ok=True, parents=True)
    
    # Infer from git if not provided
    inf = _fetch_from_git()
    repo = repo or inf.get('repo') or path.resolve().name
    user = user or inf.get('user', '')
    if not user: raise ValueError("Could not infer `user` from git. Please pass --user explicitly.")
    author = author or inf.get('author', '')
    if not author: raise ValueError("Could not infer `author` from git. Please pass --author explicitly.")
    author_email = author_email or inf.get('author_email', '')
    if not author_email: raise ValueError("Could not infer `author_email` from git. Please pass --author_email explicitly.")
    branch = branch or inf.get('branch', 'main')
    description = description or inf.get('description', '')
    
    lib_path = repo.replace('-', '_')
    git_url = f"https://github.com/{user}/{repo}" if user else ''
    doc_url = f"https://{user}.github.io/{repo}/" if user else ''
    keywords = inf.get('keywords', 'nbdev').split()
    
    txt = pyproject_tmpl.format(name=repo, lib_path=lib_path, description=description, min_python=min_python, license=license,
        author=author, author_email=author_email, keywords=keywords, git_url=git_url, doc_url=doc_url, branch=branch)
    
    cfg_file = path / pyproject_nm
    if cfg_file.exists(): warn(f'{cfg_file} already exists')
    else:
        cfg_file.write_text(txt)
        print(f'{cfg_file} created.')

You can create a `pyproject.toml` by passing settings via the command line:

```sh
nbdev_create_config --repo nbdev --user fastai --author fastai \
                    --author_email info@fast.ai --description 'A test project'
```

If you don't provide settings, we'll try to infer them from git and GitHub.

### Reading a config file -

In [ ]:
#| export
def _load_toml(p):
    "Load TOML file at `p` into a dict"
    return tomli.loads(Path(p).read_text(encoding='utf-8'))

def _has_nbdev(p):
    "True if pyproject.toml at `p` has [tool.nbdev]"
    try: return bool(_load_toml(p).get('tool', {}).get('nbdev', {}))
    except Exception: return False

def _find_nbdev_pyproject(path=None):
    "Find nearest pyproject.toml containing [tool.nbdev], walking up from `path`"
    p = Path(path or Path.cwd()).resolve()
    for d in [p] + list(p.parents):
        f = d/pyproject_nm
        if f.exists() and _has_nbdev(f): return f

In [ ]:
#| export
nbdev_defaults = dict(nbs_path='nbs', doc_path='_docs', tst_flags='notest', recursive=True, readme_nb='index.ipynb',
    clean_ids=True, clear_all=False, put_version_in_init=True, jupyter_hooks=False, black_formatting=False, branch='main')

_path_keys = 'lib_path', 'nbs_path', 'doc_path'

In [ ]:
#| export
class ConfigToml(AttrDict):
    def __init__(self, d, proj, cfg_file):
        super().__init__({**nbdev_defaults, **d})
        self.config_file = cfg_file
        self.config_path = cfg_file.parent
        
        self.lib_name = proj.get('name', '')
        self.title = self.get('title') or self.lib_name
        self.description = proj.get('description', '')
        self.keywords = proj.get('keywords', [])
        self.min_python = (proj.get('requires-python') or '>=3.9').lstrip('>=')
        auths = proj.get('authors') or [{}]
        self.author = auths[0].get('name')
        self.author_email = auths[0].get('email')
        
        
        urls = proj.get('urls') or {}
        self.git_url = (urls.get('Repository') or urls.get('Source') or '').rstrip('/')
        self.doc_url = urls.get('Documentation') or ''
        self.user, self.repo = repo_details(self.git_url) if self.git_url else ('', '')
        # Derive doc_host and doc_baseurl from doc_url
        from urllib.parse import urlparse
        u = urlparse(self.doc_url)
        self.doc_host = f"{u.scheme}://{u.netloc}" if u.scheme else ''
        self.doc_baseurl = (u.path or '/').rstrip('/') or '/'
        if 'lib_path' not in self: self['lib_path'] = self.lib_name.replace('-', '_')

    @property
    def version(self):
        return read_version(self.config_path / self['lib_path']) or '0.0.1'

    @property
    def d(self): return {k:v for k,v in super().items()}

    def __getattr__(self, k): return stop(AttributeError(k)) if k=='d' or k not in self.d else self.get(k)
    def __getitem__(self, k): return stop(IndexError(k)) if k not in self.d else self.get(k)

    def get(self, k, default=None):
        v = self.d.get(k, default)
        if v is None: return None
        return self.config_path / v if k in _path_keys else v

    def path(self, k, default=None):
        v = self.d.get(k, default)
        return v if v is None else self.config_path / v

In [ ]:
#| export
def _user_config():
    "Load user config from ~/.config/nbdev/config.toml if it exists"
    p = xdg_config_home() / _nbdev_home_dir / _user_cfg_name
    if p.exists(): return _load_toml(p)
    return {}

In [ ]:
#| export
def get_config(path=None, also_settings=False):
    "Return nbdev config."
    cfg_file = _find_nbdev_pyproject(path)
    if cfg_file is not None:
        # Check for old settings.ini and complain loudly
        old_cfg = cfg_file.parent / 'settings.ini'
        if old_cfg.exists() and not also_settings:
            raise ValueError(f"Found old settings.ini at {old_cfg}. Please migrate to pyproject.toml using `nbdev_migrate`")
        d = _load_toml(cfg_file)
        user = _user_config()
        nbdev = {**user, **d.get('tool', {}).get('nbdev', {})}
        return ConfigToml(nbdev, d.get('project', {}), cfg_file)
    if also_settings:
        from fastcore.foundation import Config
        cfg = Config.find('settings.ini', path)
        if cfg: return cfg
    cfg_path = Path(path or Path.cwd()).expanduser().absolute()
    return ConfigToml(nbdev_defaults, {}, cfg_path/'pyproject.toml')

Searches up from `path` until a `pyproject.toml` with `[tool.nbdev]` is found. Unspecified optional settings return defaults.

See `nbdev_create_config` for creating a new config.

In [ ]:
#| hide
try: _get_config,get_config = get_config,get_config.__wrapped__ # Bypass cache during dev/tests
except AttributeError: pass

In [ ]:
cfg = get_config()

`cfg` is a `ConfigToml` object (inherits from `AttrDict`), so you can access keys as attributes:

In [ ]:
p = Path.cwd().parent.parent
test_eq(cfg.lib_name, 'nbdev')
test_eq(cfg.git_url, 'https://github.com/AnswerDotAI/nbdev')

Its own path and parent are attributes too:

In [ ]:
test_eq(cfg.config_path, p)
test_eq(cfg.config_file, p/'pyproject.toml')

Paths are relative to the project:

In [ ]:
test_eq(cfg.doc_path, p/'_docs')
test_eq(cfg.lib_path, p/'nbdev')
test_eq(cfg.nbs_path, p/'nbs')

Note: If no `pyproject.toml` with `[tool.nbdev]` is found, `get_config()` returns a minimal config with defaults based on the current directory. Use `is_nbdev()` to check if you're in an nbdev project.

You can customize nbdev for all your projects by creating a `~/.config/nbdev/config.toml` file (or following the [XDG specification](https://specifications.freedesktop.org/basedir-spec/basedir-spec-latest.html)). For example, you could globally disable nbdev's Jupyter hooks with `jupyter_hooks = false`.

## Helpers

In [ ]:
#| export
def is_nbdev(path=None): return _find_nbdev_pyproject(path) is not None

In [ ]:
#| hide
test_eq(is_nbdev(), True)

In [ ]:
#| export
def create_output(txt, mime):
    "Add a cell output containing `txt` of the `mime` text MIME sub-type"
    return [dict(data={f"text/{mime}": str(txt).splitlines(True)},
                 execution_count=1, metadata={}, output_type="execute_result")]

In [ ]:
#| export
def show_src(src, lang='python'): return Markdown(f'```{lang}\n{src}\n```')

In [ ]:
show_src("print(create_output('text', 'text/plain'))")

```python
print(create_output('text', 'text/plain'))
```

##  Exporting a basic module

In [ ]:
#| export
pyproj_tmpl = """[build-system]
requires = ["setuptools>=64.0"]
build-backend = "setuptools.build_meta"

[project]
name = "FILL_IN"
requires-python="FILL_IN"
dynamic = [ "keywords", "description", "version", "dependencies", "optional-dependencies", "readme",
    "license", "authors", "classifiers", "entry-points", "scripts", "urls"]

[tool.uv]
cache-keys = [{ file = "pyproject.toml" }, { file = "setup.py" }]
"""

In [ ]:
#| export
_re_version = re.compile(r'^__version__\s*=\s*[\'"]([^\'"]+)[\'"]', re.MULTILINE)
_re_proj = re.compile(r'^name\s*=\s*".*$', re.MULTILINE)
_re_reqpy = re.compile(r'^requires-python\s*=\s*".*$', re.MULTILINE)
_init = '__init__.py'
_pyproj = 'pyproject.toml'

In [ ]:
#| export
def read_version(path):
    "Read __version__ from `path/__init__.py`, or None if not found"
    fname = Path(path)/_init
    if not fname.exists(): return None
    m = _re_version.search(fname.read_text())
    return m.group(1) if m else None

In [ ]:
#| export
def set_version(path, version):
    "Set __version__ in `path/__init__.py`"
    path = Path(path)
    path.mkdir(exist_ok=True, parents=True)
    fname = path/_init
    if not fname.exists(): fname.touch()
    ver_line = f'__version__ = "{version}"'
    code = fname.read_text()
    if _re_version.search(code) is None: code = ver_line + "\n" + code
    else: code = _re_version.sub(ver_line, code)
    fname.write_text(code)

In [ ]:
with tempfile.TemporaryDirectory() as d:
    test_is(read_version(d), None)  # No file yet
    set_version(d, '1.0.0')
    test_eq(read_version(d), '1.0.0')
    set_version(d, '1.0.1')
    test_eq(read_version(d), '1.0.1')

In [ ]:
#| export
def bump_version(v, part=2, unbump=False):
    "Bump semver string `v` at index `part` (0=major, 1=minor, 2=patch)"
    parts = (v or '0.0.0').split('.')
    parts += ['0'] * (3 - len(parts))
    parts[part] = str(int(parts[part]) + (-1 if unbump else 1))
    for i in range(part+1, 3): parts[i] = '0'
    return '.'.join(parts[:3])

In [ ]:
#| export
def update_version(path=None):
    "Add __version__ to `path/__init__.py` if it doesn't exist"
    path = Path(path or get_config().lib_path)
    if read_version(path) is None: set_version(path, get_config().version)

In [ ]:
test_eq(bump_version('1.2.3'), '1.2.4')
test_eq(bump_version('1.2.3', part=1), '1.3.0')
test_eq(bump_version('1.2.3', part=0), '2.0.0')
test_eq(bump_version('1.2.3', part=2, unbump=True), '1.2.2')

In [ ]:
#| export
def _has_py(fs): return any(1 for f in fs if f.endswith('.py'))

def update_proj(path):
    "Create or update `pyproject.toml` in the project root."
    fname = path/_pyproj
    if not fname.exists(): fname.write_text(pyproj_tmpl)
    txt = fname.read_text()
    txt = _re_proj.sub(f'name = "{get_config().lib_name}"', txt)
    txt = _re_reqpy.sub(f'requires-python = ">={get_config().min_python}"', txt)
    fname.write_text(txt)

In [ ]:
#| export
def add_init(path=None):
    "Add `__init__.py` in all subdirs of `path` containing python files if it's not there already."
    # we add the lowest-level `__init__.py` files first, which ensures _has_py succeeds for parent modules
    path = Path(path or get_config().lib_path)
    path.mkdir(exist_ok=True)
    if not (path/_init).exists(): (path/_init).touch()
    for r,ds,fs in os.walk(path, topdown=False):
        r = Path(r)
        subds = (os.listdir(r/d) for d in ds)
        if _has_py(fs) or any(filter(_has_py, subds)) and not (r/_init).exists(): (r/_init).touch()
    if get_config().get('put_version_in_init', True): update_version(path)
    if get_config().get('update_pyproject', True): update_proj(path.parent)

Python modules require a `__init.py__` file in all directories that are modules. We assume that all directories containing a python file (including in subdirectories of any depth) is a module, and therefore add a `__init__.py` to each.

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    (d/'a/b').mkdir(parents=True)
    (d/'a/b/f.py').touch()
    (d/'a/c').mkdir()
    add_init(d)
    assert not (d/'a/c'/_init).exists(), "Should not add init to dir without py file"
    for e in [d, d/'a', d/'a/b']: assert (e/_init).exists(),f"Missing init in {e}"

In [ ]:
#| export
def write_cells(cells, hdr, file, solo_nb=False):
    "Write `cells` to `file` along with header `hdr` (mainly for nbdev internal use)."
    for cell in cells:
        if cell.cell_type=='code' and cell.source.strip():
            cell_id = f" #{cell.id}" if cell.get('id') else ""
            file.write(f'\n\n{hdr}{cell_id}\n{cell.source}') if not solo_nb else file.write(f'\n\n{cell.source}')

In [ ]:
#| export
def _basic_export_nb(fname, name, dest=None):
    "Basic exporter to bootstrap nbdev."
    if dest is None: dest = get_config().lib_path
    add_init()
    fname,dest = Path(fname),Path(dest)
    nb = read_nb(fname)

    # grab the source from all the cells that have an `export` comment
    cells = L(cell for cell in nb.cells if re.match(r'#\s*\|export', cell.source))

    # find all the exported functions, to create `__all__`:
    trees = cells.map(NbCell.parsed_).concat()
    funcs = trees.filter(risinstance((ast.FunctionDef,ast.ClassDef))).attrgot('name')
    exp_funcs = [f for f in funcs if f[0]!='_']

    # write out the file
    with (dest/name).open('w',encoding="utf-8") as f:
        f.write(f"# %% auto 0\n__all__ = {exp_funcs}")
        write_cells(cells, f"# %% {fname.relpath(dest)}", f)
        f.write('\n')

This is a simple exporter with just enough functionality to correctly export this notebook, in order to bootstrap the creation of nbdev itself.

In [ ]:
# #| hide
# #| eval: false
# path = Path('../nbdev')
# (path/'config.py').unlink(missing_ok=True)
# 
# _basic_export_nb("01_config.ipynb", 'config.py')
# 
# g = exec_new('from nbdev import config')
# assert g['config'].add_init
# assert 'add_init' in g['config'].__all__